# **Бот рейтинга**
![alt text](rating.jpg)

**Меню для BotFather:**   
component - сообщить текущий рейтинг

In [26]:
# user_id - НЕ РАЗВИВАЕТСЯ

# PersonalRatingBot @PersonalRatingBot
import pandas as pd
import numpy as np
import os
import sys
import sqlite3
import telebot
from telebot import types

# from requests.exceptions import ConnectionError

# import time
import datetime
from datetime import datetime

# Токен
TOKEN = "7350611722:AAGP6sVlNpkBfxS1p-Vv4pHTYscOPPdgGW8"  # token PersonalRatingBot

messages = {}
password_flag = {}
db_path = os.getcwd().replace("\\", "/") + "/db_order.sqlite"

# пароль администратора для управления списком избранных
admin_password = "000"

# Создание экземпляра бота
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=["component"])
def start(message):
    """меню бота"""
    global act, messages, elite_users, admin_password
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    query_test = "SELECT * FROM user_status us WHERE us.user_id = ?"
    if len(cur.execute(query_test, (user_id,)).fetchall()) == 0:
        query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
        cur.execute(
            query_insert,
            (
                user_id,
                user_id,
                user_datatime,
                0,
            ),
        )
        conn.commit()

    query_m = """SELECT us.user_id FROM user_status us WHERE us.flag = 1;"""
    df_elite_users = pd.DataFrame(cur.execute(query_m).fetchall(), columns=["user_id"])
    try:
        elite_users = df_elite_users["user_id"].tolist()
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")

    if user_id in elite_users:
        try:
            bot.send_message(
                message.chat.id,
                text=f"""
            {message.from_user.first_name}, в личные сообщения направлен ваш текущий рейтинг по состоянию на {user_datatime}""",
                parse_mode="Markdown",
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Глобальный трабл 1, ибо {e}""")
        try:
            bot.send_message(
                message.from_user.id, f"Ваш рейтинг {message.from_user.id}"
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")
    else:
        bot.send_message(
            message.chat.id,
            f"""{message.from_user.first_name}, Вы не сотрудник КЦ Минздрава или не член рабочей группы региона - эта опция для Вас не доступна""",
        )

@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global act, messages, elite_users, admin_password
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    messages[user_id] = message.text
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    try:
        if messages.get(user_id) == admin_password:
            bot.delete_message(message.chat.id, message.message_id)
            password_flag[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("👻Добавить Избранного")
            btn2 = types.KeyboardButton("☠️Прогнать Избранного")
            markup.add(btn1, btn2)
            bot.send_message(
                message.chat.id,
                f"Что Вы хотите сделать, босс?",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "👻Добавить Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""💥Отправьте id нового Избранного""")
            act = 1
        elif messages.get(user_id) == "☠️Прогнать Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""⚰️Отправьте id обречённого""")
            act = 0
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 1
        ):
            query_kill = "DELETE FROM user_status WHERE user_id = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.id, user_datatime, 1),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 0
        ):
            query_kill = "DELETE FROM user_status WHERE user_id = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.id, user_datatime, 0),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Глобальный трабл 2, ибо {e}""")


# запуск бота
bot.infinity_polling()

2024-08-20 17:36:22,926 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-08-20 17:36:22,926 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [8]:
# username 2.0 - без ссылки на бот с личными сообщениями - НЕ РАЗВИВАЕТСЯ

# PersonalRatingBot @PersonalRatingBot
import pandas as pd
import numpy as np
import os
import sqlite3
import telebot
from telebot import types

# from requests.exceptions import ConnectionError

# import time
import datetime
from datetime import datetime

# Токен
# TOKEN = "7350611722:AAGP6sVlNpkBfxS1p-Vv4pHTYscOPPdgGW8"  # token PersonalRatingBot
TOKEN = "7213028141:AAGztt3IBI8wDipz_gmXmbNhVagpnDqfwzU"  # token BigTestingDotNew (Цветочек)

messages = {}
password_flag = {}
db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"

# пароль администратора для управления списком избранных
admin_password = "000"

# Создание экземпляра бота
bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=["start"])  # в правильной версии надо component
def start(message):
    """меню бота"""
    global act, messages, elite_users, admin_password
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    username = message.from_user.username
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    query_test = "SELECT * FROM user_status us WHERE us.user_name = ?"
    if len(cur.execute(query_test, (username,)).fetchall()) == 0:
        query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
        cur.execute(
            query_insert,
            (
                username,
                "auto",
                user_datatime,
                0,
            ),
        )
        conn.commit()

    query_m = """SELECT us.user_name FROM user_status us WHERE us.flag = 1;"""
    df_elite_users = pd.DataFrame(
        cur.execute(query_m).fetchall(), columns=["user_name"]
    )
    try:
        elite_users = df_elite_users["user_name"].tolist()
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")

    if username in elite_users:
        try:
            bot.send_message(
                message.chat.id,
                text=f"""
            {message.from_user.first_name}, в личные сообщения направлен ваш текущий рейтинг по состоянию на {user_datatime}""",
                parse_mode="Markdown",
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Глобальный трабл 1, ибо {e}""")
        try:
            bot.send_message(
                message.from_user.id, f"Ваш рейтинг {message.from_user.id}"
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")
    else:
        bot.send_message(
            message.chat.id,
            f"""{message.from_user.first_name}, Вы не сотрудник КЦ Минздрава или не член рабочей группы региона - эта опция для Вас не доступна""",
        )


@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global act, messages, elite_users, admin_password
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    username = message.from_user.username
    messages[user_id] = message.text
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    try:
        if messages.get(user_id) == admin_password:
            bot.delete_message(message.chat.id, message.message_id)
            password_flag[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("👻Добавить Избранного")
            btn2 = types.KeyboardButton("☠️Прогнать Избранного")
            markup.add(btn1, btn2)
            bot.send_message(
                message.chat.id,
                f"Что Вы хотите сделать, босс?",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "👻Добавить Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(
                message.chat.id, f"""💥Отправьте username нового Избранного"""
            )
            act = 1
        elif messages.get(user_id) == "☠️Прогнать Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""⚰️Отправьте username обречённого""")
            act = 0
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 1
        ):
            query_kill = "DELETE FROM user_status WHERE user_name = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.username, user_datatime, 1),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 0
        ):
            query_kill = "DELETE FROM user_status WHERE user_name = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.username, user_datatime, 0),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Глобальный трабл 2, ибо {e}""")


# запуск бота
bot.infinity_polling()

2024-08-21 14:57:53,845 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-08-21 14:57:53,847 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


![alt text](цветочег150.jpg)

In [56]:
# username 2.1 - + ссылка для перехода в бот с личными сообщениями - РАЗВИВАЕТСЯ

# PersonalRatingBot @PersonalRatingBot
import pandas as pd
import numpy as np
import os
import sqlite3
import telebot
from telebot import types

# from requests.exceptions import ConnectionError

# import time
import datetime
from datetime import datetime

# Токен
# TOKEN = "7350611722:AAGP6sVlNpkBfxS1p-Vv4pHTYscOPPdgGW8"  # token PersonalRatingBot
TOKEN = "7213028141:AAGztt3IBI8wDipz_gmXmbNhVagpnDqfwzU"  # token BigTestingDotNew (Цветочек)

messages = {}
password_flag = {}
password_flag_add = {}
act = {}
act_reg = {}
region_edit = {}
db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"

# пароль администратора для управления списком избранных
admin_password = "000"

# пароль администратиора для сопоставления юзера и региона
userreg_password = "111"

# пароль администратиора для обновления БД
update_password = "999"

# список правильных названий регионов
conn = sqlite3.connect(db_path)
cur = conn.cursor()
query_regions = """
SELECT rn.reg_name_rus_etalon 
FROM regions_names rn;
"""
etalon_regnames_list = [x[0] for x in cur.execute(query_regions).fetchall()]


def update():
    """обновление БД"""
    global data_path, last_data_path
    data_file_list = os.listdir(data_path)
    source_files_dict = {}
    for filename in data_file_list:
        if filename.split(".")[-1] == "xlsx":
            source_files_dict[os.path.getmtime(data_path + filename)] = filename
    last_data_path = data_path + source_files_dict.get(max(source_files_dict.keys()))
    try:
        df_data = pd.read_excel(
            last_data_path,
            sheet_name="Лист1",
            index_col=None,
        )
    except Exception as e:
        sys.exit(e)
    # постоянная часть
    df_constant = df_data[df_data.columns[[0, 1, 2, 5, 7, 8, 10, 12, 14]]]
    constatn_columns = [
        "id",
        "rating",
        "region_name",
        "total",
        "total_semd",
        "ouz",
        "ouz_semd",
        "private",
        "private_semd",
    ]
    df_constant.columns = constatn_columns
    df_constant = df_constant.drop(index=[0, 1, 2, 3])
    df_constant = df_constant.reset_index(drop=True)
    df_constant = df_constant.astype(
        {
            "id": int,
            "rating": int,
            "total": int,
            "total_semd": int,
            "ouz": int,
            "ouz_semd": int,
            "private": int,
            "private_semd": int,
        }
    )
    # динамическая часть
    variable_columns_numbers = [2] + [x for x in range(44, df_data.shape[1])]
    df_variable = df_data[df_data.columns[variable_columns_numbers]]
    df_variable = df_variable.drop(index=[0, 1, 2])
    df_variable.at[3, df_variable.columns[0]] = "region_name"
    df_variable.columns = df_variable.iloc[0]
    df_variable = df_variable.drop([3]).reset_index(drop=True)
    df_variable = pd.melt(
        df_variable,
        id_vars=df_variable.columns[0],
        value_vars=[df_variable.columns[x] for x in range(1, df_variable.shape[1])],
    )
    df_variable = df_variable.rename(columns={3: "date_ratio"})
    df_variable["value"] = df_variable["value"].astype("float")
    # заливка с заменой
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    df_constant.to_sql("semd_status", con=conn, index=False, if_exists="replace")
    df_variable.to_sql("semd_dynamic", con=conn, index=False, if_exists="replace")
    return last_data_path


#  формирование ответа
def answerer(user_name):
    """получение ответа из db, один пользователь может представлять более одного региона"""
    global db_path, answer_array
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query = """
	SELECT
		rn.reg_name_rus_etalon,
		ss.region_name AS reg_name_rating,
		ss.rating,
		COALESCE (ss.total_semd*100.0/ss.total, 0) AS total_ratio,
		COALESCE (ss.private_semd*100.0/ss.private, 0) AS private_ratio,
		COALESCE (ss.private-ss.private_semd, 0) AS private_nosemd,
		COALESCE (ss.ouz_semd*100.0/ss.ouz, 0) AS ouz_ratio,
		COALESCE (ss.ouz-ss.ouz_semd, 0) AS ouz_nosemd,
		sd.report_date
	FROM semd_status ss
	LEFT JOIN
	(
		SELECT sd.region_name, date(MAX(sd.date_ratio)) AS report_date
		FROM semd_dynamic sd
		GROUP BY sd.region_name
	) AS sd ON ss.region_name = sd.region_name
	LEFT JOIN regions_names rn
	ON rn.reg_name_rus_rating = ss.region_name
	WHERE rn.reg_name_rus_etalon IN 
	(
		SELECT ur.reg_name_rus_etalon
		FROM user_region ur
		WHERE ur.user_name = ?
	);
	"""
    query_max_rating = """
	SELECT MAX(rating)
	FROM semd_status ss;
	"""
    max_rating = cur.execute(query_max_rating).fetchall()[0][0]

    try:
        answer_array = cur.execute(query, (user_name,)).fetchall()
        answer = ""
        for i in range(len(answer_array)):
            answer_tmp = f"""
	✅Уважаемый *{user_name}*:
		• ваш регион: *{answer_array[i][0]}*;
		• отчётная дата: *{answer_array[i][8]}*;
		• рейтинг региона: *{answer_array[i][2]}* (из *{max_rating}*);
		• доля МО, передавших не менее одного СЭМД: *{round(answer_array[i][3], 1)} %*;
		• доля частных МО, передавших не менее одного СЭМД: *{round(answer_array[i][4], 1)} %*;
		• количество частных МО, не передавших ни одного СЭМД: *{"нет" if answer_array[i][5] == 0 else answer_array[i][5]}*;
		• доля МО, подвед. ОУЗ, передавших не менее одного СЭМД: *{round(answer_array[i][6], 1)} %*;
		• количество МО, подвед. ОУЗ, не передавших ни одного СЭМД: *{"нет" if answer_array[i][7] == 0 else answer_array[i][7]}*.
                            
		"""
            answer += answer_tmp
    except Exception:
        answer = f"""
        🚫Уважаемый *{user_name}*, для Вас информация недоступна, обратитесь к администратору чат-бота."""
    return answer


# Создание экземпляра бота
bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=["start"])  # в правильной версии надо component
def start(message):
    """меню бота"""
    global act, act_reg, messages, elite_users, admin_password, userreg_password, region_edit
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    username = message.from_user.username
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    query_test = "SELECT * FROM user_status us WHERE us.user_name = ?"
    if len(cur.execute(query_test, (username,)).fetchall()) == 0:
        query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
        cur.execute(
            query_insert,
            (
                username,
                "auto",
                user_datatime,
                0,
            ),
        )
        conn.commit()

    query_m = """SELECT us.user_name FROM user_status us WHERE us.flag = 1;"""
    df_elite_users = pd.DataFrame(
        cur.execute(query_m).fetchall(), columns=["user_name"]
    )
    try:
        elite_users = df_elite_users["user_name"].tolist()
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Локальный сбой, ибо {e}""")

    if username in elite_users:
        try:
            markup = types.InlineKeyboardMarkup()
            btn1 = types.InlineKeyboardButton(
                "перейдите для получения рейтинга", url="https://t.me/PersonalRatingBot"
            )
            markup.add(btn1)
            bot.send_message(
                message.chat.id,
                f"{message.from_user.first_name}, в личные сообщения направлен ваш текущий рейтинг по состоянию на {user_datatime}.",
                reply_markup=markup,
                parse_mode="Markdown",
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Глобальный сбой 1, ибо {e}""")
        try:
            answer = answerer(username)
            bot.send_message(message.from_user.id, answer, parse_mode="Markdown")
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Локальный сбой, ибо {e}""")
    else:
        bot.send_message(
            message.chat.id,
            f"""{message.from_user.first_name}, Вы не сотрудник КЦ Минздрава или не член рабочей группы региона - эта опция для Вас не доступна.""",
        )


@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global act, ast_reg, messages, elite_users, admin_password, userreg_password, region_edit
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    username = message.from_user.username
    messages[user_id] = message.text
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    try:
        if messages.get(user_id) == admin_password:
            bot.delete_message(message.chat.id, message.message_id)
            password_flag[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("👻Добавить Избранного")
            btn2 = types.KeyboardButton("☠️Прогнать Избранного")
            btn3 = types.KeyboardButton("Вернуться в чат")
            markup.add(btn1, btn2, btn3)
            bot.send_message(
                message.chat.id,
                f"Что Вы хотите сделать, босс?",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "👻Добавить Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(
                message.chat.id, f"""💥Отправьте username нового Избранного"""
            )
            act[user_id] = 1
        elif messages.get(user_id) == "☠️Прогнать Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""⚰️Отправьте username обречённого""")
            act[user_id] = 0
        elif messages.get(user_id) == "Вернуться в чат" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag[user_id] = False
            bot.send_message(message.chat.id, "👍")           
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act.get(user_id) == 1
        ):
            query_kill = "DELETE FROM user_status WHERE user_name = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.username, user_datatime, 1),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act.get(user_id) == 0
        ):
            query_kill = "DELETE FROM user_status WHERE user_name = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.username, user_datatime, 0),
            )
            conn.commit()
            act[user_id] = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")

        elif messages.get(user_id) == userreg_password:
            bot.delete_message(message.chat.id, message.message_id)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Дальневосточный")
            btn2 = types.KeyboardButton("Приволжский")
            btn3 = types.KeyboardButton("Северо-Западный")
            btn4 = types.KeyboardButton("Северо-Кавказский")
            btn5 = types.KeyboardButton("Сибирский")
            btn6 = types.KeyboardButton("Уральский")
            btn7 = types.KeyboardButton("Центральный")
            btn8 = types.KeyboardButton("Южный")
            markup.add(btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8)
            bot.send_message(
                message.chat.id,
                f"Выберете федеральный округ:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Дальневосточный" and password_flag_add.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Амурская область")
            btn2 = types.KeyboardButton("Еврейская автономная область")
            btn3 = types.KeyboardButton("Забайкальский край")
            btn4 = types.KeyboardButton("Камчатский край")
            btn5 = types.KeyboardButton("Магаданская область")
            btn6 = types.KeyboardButton("Приморский край")
            btn7 = types.KeyboardButton("Республика Бурятия")
            btn8 = types.KeyboardButton("Республика Саха (Якутия)")
            btn9 = types.KeyboardButton("Сахалинская область")
            btn10 = types.KeyboardButton("Хабаровский край")
            btn11 = types.KeyboardButton("Чукотский автономный округ")
            markup.add(
                btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8, btn9, btn10, btn11
            )
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Приволжский" and password_flag_add.get(user_id):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Кировская область")
            btn2 = types.KeyboardButton("Нижегородская область")
            btn3 = types.KeyboardButton("Оренбургская область")
            btn4 = types.KeyboardButton("Пензенская область")
            btn5 = types.KeyboardButton("Пермский край")
            btn6 = types.KeyboardButton("Республика Башкортостан")
            btn7 = types.KeyboardButton("Республика Марий Эл")
            btn8 = types.KeyboardButton("Республика Мордовия")
            btn9 = types.KeyboardButton("Республика Татарстан")
            btn10 = types.KeyboardButton("Самарская область")
            btn11 = types.KeyboardButton("Саратовская область")
            btn12 = types.KeyboardButton("Удмуртская Республика")
            btn13 = types.KeyboardButton("Ульяновская область")
            btn14 = types.KeyboardButton("Чувашская Республика")
            markup.add(
                btn1,
                btn2,
                btn3,
                btn4,
                btn5,
                btn6,
                btn7,
                btn8,
                btn9,
                btn10,
                btn11,
                btn12,
                btn13,
                btn14,
            )
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Северо-Западный" and password_flag_add.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Архангельская область")
            btn2 = types.KeyboardButton("Вологодская область")
            btn3 = types.KeyboardButton("г. Санкт-Петербург")
            btn4 = types.KeyboardButton("Калининградская область")
            btn5 = types.KeyboardButton("Ленинградская область")
            btn6 = types.KeyboardButton("Мурманская область")
            btn7 = types.KeyboardButton("Ненецкий автономный округ")
            btn8 = types.KeyboardButton("Новгородская область")
            btn9 = types.KeyboardButton("Псковская область")
            btn10 = types.KeyboardButton("Республика Карелия")
            btn11 = types.KeyboardButton("Республика Коми")
            markup.add(
                btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8, btn9, btn10, btn11
            )
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Северо-Кавказский" and password_flag_add.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Кабардино-Балкарская Республика")
            btn2 = types.KeyboardButton("Карачаево-Черкесская Республика")
            btn3 = types.KeyboardButton("Республика Дагестан")
            btn4 = types.KeyboardButton("Республика Ингушетия")
            btn5 = types.KeyboardButton("Республика Северная Осетия - Алания")
            btn6 = types.KeyboardButton("Ставропольский край")
            btn7 = types.KeyboardButton("Чеченская Республика")
            markup.add(btn1, btn2, btn3, btn4, btn5, btn6, btn7)
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Сибирский" and password_flag_add.get(user_id):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Алтайский край")
            btn2 = types.KeyboardButton("Иркутская область")
            btn3 = types.KeyboardButton("Кемеровская область")
            btn4 = types.KeyboardButton("Красноярский край")
            btn5 = types.KeyboardButton("Новосибирская область")
            btn6 = types.KeyboardButton("Омская область")
            btn7 = types.KeyboardButton("Республика Алтай")
            btn8 = types.KeyboardButton("Республика Тыва")
            btn9 = types.KeyboardButton("Республика Хакасия")
            btn10 = types.KeyboardButton("Томская область")
            markup.add(btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8, btn9, btn10)
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Уральский" and password_flag_add.get(user_id):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Курганская область")
            btn2 = types.KeyboardButton("Свердловская область")
            btn3 = types.KeyboardButton("Тюменская область")
            btn4 = types.KeyboardButton("Ханты-Мансийский автономный округ - Югра")
            btn5 = types.KeyboardButton("Челябинская область")
            btn6 = types.KeyboardButton("Ямало-Ненецкий автономный округ")
            markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Центральный" and password_flag_add.get(user_id):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Белгородская область")
            btn2 = types.KeyboardButton("Брянская область")
            btn3 = types.KeyboardButton("Владимирская область")
            btn4 = types.KeyboardButton("Воронежская область")
            btn5 = types.KeyboardButton("г. Москва")
            btn6 = types.KeyboardButton("Ивановская область")
            btn7 = types.KeyboardButton("Калужская область")
            btn8 = types.KeyboardButton("Костромская область")
            btn9 = types.KeyboardButton("Курская область")
            btn10 = types.KeyboardButton("Липецкая область")
            btn11 = types.KeyboardButton("Московская область")
            btn12 = types.KeyboardButton("Орловская область")
            btn13 = types.KeyboardButton("Рязанская область")
            btn14 = types.KeyboardButton("Смоленская область")
            btn15 = types.KeyboardButton("Тамбовская область")
            btn16 = types.KeyboardButton("Тверская область")
            btn17 = types.KeyboardButton("Тульская область")
            btn18 = types.KeyboardButton("Ярославская область")
            markup.add(
                btn1,
                btn2,
                btn3,
                btn4,
                btn5,
                btn6,
                btn7,
                btn8,
                btn9,
                btn10,
                btn11,
                btn12,
                btn13,
                btn14,
                btn15,
                btn16,
                btn17,
                btn18,
            )
            bot.send_message(
                message.chat.id,
                f"Выберете субъект Российской Федерации:",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "Южный" and password_flag_add.get(user_id):
            bot.delete_message(message.chat.id, message.message_id)
            bot.delete_message(message.chat.id, message.message_id - 1)
            password_flag_add[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Астраханская область")
            btn2 = types.KeyboardButton("Волгоградская область")
            btn3 = types.KeyboardButton("г. Севастополь")
            btn4 = types.KeyboardButton("Краснодарский край")
            btn5 = types.KeyboardButton("Республика Адыгея")
            btn6 = types.KeyboardButton("Республика Калмыкия")
            btn7 = types.KeyboardButton("Республика Крым")
            btn8 = types.KeyboardButton("Ростовская область")
            markup.add(btn1, btn2, btn3, btn4, btn5, btn6, btn7, btn8)
            bot.send_message(
                message.chat.id,
                f"Выберете регион:",
                reply_markup=markup,
            )
        elif messages.get(user_id) in etalon_regnames_list and password_flag_add.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id)
            region_edit[user_id] = messages.get(user_id)
            query_users_region = """
            SELECT ur.user_name 
            FROM user_region ur
            WHERE ur.reg_name_rus_etalon = ?;
            """
            users_list = [
                x[0]
                for x in cur.execute(
                    query_users_region, (messages.get(user_id),)
                ).fetchall()
            ]
            if len(users_list) == 0:
                answer_user_region = "здесь пока никого нет"
            else:
                answer_user_region = "\n• ".join(users_list)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("Добавить пользователя в регион")
            btn2 = types.KeyboardButton("Удалить пользователя из региона")
            btn3 = types.KeyboardButton("Вернуться в чат")
            markup.add(btn1, btn2, btn3)
            bot.send_message(
                message.chat.id,
                f"""В регионе *{messages.get(user_id)}* данные доступны для:
• {answer_user_region}""",
                reply_markup=markup,
                parse_mode="Markdown",
            )
        elif messages.get(user_id) == "Вернуться в чат" and password_flag_add.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
            password_flag_add[user_id] = False
        elif messages.get(
            user_id
        ) == "Добавить пользователя в регион" and password_flag_add.get(user_id):
            act_reg[user_id] = 1
            bot.delete_message(message.chat.id, message.message_id)
            bot.send_message(
                message.chat.id,
                f"Укажите username пользователя для добавления в регион *{region_edit.get(user_id)}*",
                parse_mode="Markdown",
            )
        elif messages.get(
            user_id
        ) == "Удалить пользователя из региона" and password_flag_add.get(user_id):
            act_reg[user_id] = 0
            bot.delete_message(message.chat.id, message.message_id)
            bot.send_message(
                message.chat.id,
                f"Укажите username пользователя для удаления из региона *{region_edit.get(user_id)}*",
                parse_mode="Markdown",
            )
        elif (
            messages.get(user_id)
            and password_flag_add.get(user_id)
            and act_reg.get(user_id) == 1
        ):
            # добавление юзера в регион
            bot.delete_message(message.chat.id, message.message_id)
            query_test_user = """
            SELECT ur.reg_name_rus_etalon
            FROM user_region ur
            WHERE ur.user_name = ?;
            """
            user_result = cur.execute(
                query_test_user, (messages.get(user_id),)
            ).fetchall()
            if len(user_result) != 0:
                regions_str = ", ".join([x[0] for x in user_result])
                bot.send_message(
                    message.chat.id,
                    f"Пользователь *{messages.get(user_id)}* уже добавлен в регионы *{regions_str}*, а теперь добавится ещё и в регион {region_edit.get(user_id)}.",
                    parse_mode="Markdown",
                )
            query_add = """
            INSERT INTO user_region (reg_name_rus_etalon, user_name) VALUES (?, ?);
            """
            cur.execute(
                query_add,
                (
                    region_edit.get(user_id),
                    messages.get(user_id),
                ),
            )
            query_kill = "DELETE FROM user_status WHERE user_name = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            query_insert = "INSERT INTO user_status (user_name, admin_name, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (
                    messages.get(user_id),
                    message.from_user.username,
                    user_datatime,
                    1,
                ),
            )
            conn.commit()
            bot.send_message(
                message.chat.id,
                f"✅Пользователь *{messages.get(user_id)}* успешно добавлен в  регион *{region_edit.get(user_id)}*, активирована отправка отчётов.",
                parse_mode="Markdown",
            )
            act_reg[user_id] = 3
        elif (
            messages.get(user_id)
            and password_flag_add.get(user_id)
            and act_reg.get(user_id) == 0
        ):
            # удаление юзера из региона
            bot.delete_message(message.chat.id, message.message_id)
            query_kill_r = """
            DELETE FROM user_region WHERE reg_name_rus_etalon = ? AND user_name = ?
            """
            cur.execute(
                query_kill_r,
                (
                    region_edit.get(user_id),
                    messages.get(user_id),
                ),
            )
            conn.commit()
            bot.send_message(
                message.chat.id,
                f"✅Пользователь *{messages.get(user_id)}* успешно удалён из региона *{region_edit.get(user_id)}*, при необходимости не забудьте дезактивировать отправку отчётов.",
                parse_mode="Markdown",
            )
            act_reg[user_id] = 3
        elif messages.get(user_id) == update_password:
            bot.delete_message(message.chat.id, message.message_id)
            update()
            update_answer = str(update())
            bot.send_message(
                message.chat.id,
                f"❗️Загружены данные из файла {update_answer}",
            )
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Глобальный сбой 2, ибо {e}""")


# запуск бота
bot.infinity_polling()

2024-08-30 18:12:07,618 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-08-30 18:12:07,618 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [48]:
# PersonalRatingBot @PersonalRatingBot
import pandas as pd
import numpy as np
import os
import sqlite3
import telebot
from telebot import types

# from requests.exceptions import ConnectionError

# import time
import datetime
from datetime import datetime

db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"

def answerer(user_name):
    """получение ответа из db
    один пользователь может представлять более одного региона"""
    global db_path, answer_array
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    query = """
	SELECT
		rn.reg_name_rus_etalon,
		ss.region_name AS reg_name_rating,
		ss.rating,
		COALESCE (ss.total_semd*100.0/ss.total, 0) AS total_ratio,
		COALESCE (ss.private_semd*100.0/ss.private, 0) AS private_ratio,
		COALESCE (ss.private-ss.private_semd, 0) AS private_nosemd,
		COALESCE (ss.ouz_semd*100.0/ss.ouz, 0) AS ouz_ratio,
		COALESCE (ss.ouz-ss.ouz_semd, 0) AS ouz_nosemd,
		sd.report_date
	FROM semd_status ss
	LEFT JOIN
	(
		SELECT sd.region_name, date(MAX(sd.date_ratio)) AS report_date
		FROM semd_dynamic sd
		GROUP BY sd.region_name
	) AS sd ON ss.region_name = sd.region_name
	LEFT JOIN regions_names rn
	ON rn.reg_name_rus_rating = ss.region_name
	WHERE rn.reg_name_rus_etalon IN 
	(
		SELECT ur.reg_name_rus_etalon
		FROM user_region ur
		WHERE ur.user_name = ?
	);
	"""
    query_max_rating = """
	SELECT MAX(rating)
	FROM semd_status ss;
	"""
    max_rating = cur.execute(query_max_rating).fetchall()[0][0]

    try:
        answer_array = cur.execute(query, (user_name,)).fetchall()
        answer = ""
        for i in range(len(answer_array)):
            answer_tmp = f"""
	✅Уважаемый *{user_name}*:
		• ваш регион: *{answer_array[i][0]}*;
		• отчётная дата: *{answer_array[i][8]}*;
		• рейтинг региона: *{answer_array[i][2]}* (из *{max_rating}*);
		• доля МО, передавших не менее одного СЭМД: *{round(answer_array[i][3], 1)} %*;
		• доля частных МО, передавших не менее одного СЭМД: *{round(answer_array[i][4], 1)} %*;
		• количество частных МО, не передавших ни одного СЭМД: *{"нет" if answer_array[i][5] == 0 else answer_array[i][5]}*;
		• доля МО, подвед. ОУЗ, передавших не менее одного СЭМД: *{round(answer_array[i][6], 1)} %*;
		• количество МО, подвед. ОУЗ, не передавших ни одного СЭМД: *{"нет" if answer_array[i][7] == 0 else answer_array[i][7]}*.
                            
		"""
            answer+=answer_tmp
    except Exception:
        answer = f"""
        🚫Уважаемый *{user_name}*, для Вас информация недоступна, обратитесь к администратору чат-бота."""
    return answer

user_name = "kinick"
print(answerer(user_name))


	✅Уважаемый *kinick*:
		• ваш регион: *Ивановская область*;
		• отчётная дата: *2024-08-28*;
		• рейтинг региона: *54* (из *85*);
		• доля МО, передавших не менее одного СЭМД: *77.6 %*;
		• доля частных МО, передавших не менее одного СЭМД: *57.1 %*;
		• количество частных МО, не передавших ни одного СЭМД: *9*;
		• доля МО, подвед. ОУЗ, передавших не менее одного СЭМД: *100.0 %*;
		• количество МО, подвед. ОУЗ, не передавших ни одного СЭМД: *нет*.
                            
		


In [33]:
answer_array

[('Ивановская область',
  'Ивановская область',
  54,
  77.55102040816327,
  57.142857142857146,
  9,
  100.0,
  0,
  '2024-08-22')]

In [7]:
str_1 = """Тестовая строка:
- допустим, 1;
- допустим, 2
"""

str_2 = """Тестовая строка:
- допустим, 3;
- допустим, 4...
"""

print(str_1 + str_2, end="\n")

Тестовая строка:
- допустим, 1;
- допустим, 2
Тестовая строка:
- допустим, 3;
- допустим, 4...



## Получение данных

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import sqlite3
from datetime import datetime

In [27]:
data_path = os.getcwd().replace("\\", "/") + "/source_data/"
data_file_list = os.listdir(data_path)
source_files_dict = {}
for filename in data_file_list:
    if filename.split(".")[-1] == "xlsx":
        source_files_dict[os.path.getmtime(data_path + filename)] = filename
last_data_path = data_path + source_files_dict.get(max(source_files_dict.keys()))
try:
    df_data = pd.read_excel(
        last_data_path,
        sheet_name="Лист1",
        index_col=None,
    )
except Exception as e:
    sys.exit(e)
print(f"используется файл:\t{last_data_path}")

# постоянная часть
df_constant = df_data[df_data.columns[[0, 1, 2, 5, 7, 8, 10, 12, 14]]]
constatn_columns = [
    "id",
    "rating",
    "region_name",
    "total",
    "total_semd",
    "ouz",
    "ouz_semd",
    "private",
    "private_semd",
]
df_constant.columns = constatn_columns
df_constant = df_constant.drop(index=[0, 1, 2, 3])
df_constant = df_constant.reset_index(drop=True)
df_constant = df_constant.astype(
    {
        "id": int,
        "rating": int,
        "total": int,
        "total_semd": int,
        "ouz": int,
        "ouz_semd": int,
        "private": int,
        "private_semd": int,
    }
)

# динамическая часть
variable_columns_numbers = [2] + [x for x in range(44, df_data.shape[1])]
df_variable = df_data[df_data.columns[variable_columns_numbers]]
df_variable = df_variable.drop(index=[0, 1, 2])
df_variable.at[3, df_variable.columns[0]] = "region_name"
df_variable.columns = df_variable.iloc[0]
df_variable = df_variable.drop([3]).reset_index(drop=True)
df_variable = pd.melt(
    df_variable,
    id_vars=df_variable.columns[0],
    value_vars=[df_variable.columns[x] for x in range(1, df_variable.shape[1])],
)
df_variable = df_variable.rename(columns={3: "date_ratio"})
df_variable["value"] = df_variable["value"].astype("float")

rating_date = df_variable["date_ratio"].max().strftime("%Y-%m-%d")

используется файл:	c:/work/pkz_bot/bot_order/source_data/Рейтинг_И7_22.08.2024 в1.xlsx


In [28]:
# заливка с заменой
db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"
conn = sqlite3.connect(db_path)
cur = conn.cursor()
df_constant.to_sql("semd_status", con=conn, index=False, if_exists="replace")
df_variable.to_sql("semd_dynamic", con=conn, index=False, if_exists="replace")

680

In [45]:
def update():
    """обновление БД"""
    data_path = os.getcwd().replace("\\", "/") + "/source_data/"
    data_file_list = os.listdir(data_path)
    source_files_dict = {}
    for filename in data_file_list:
        if filename.split(".")[-1] == "xlsx":
            source_files_dict[os.path.getmtime(data_path + filename)] = filename
    last_data_path = data_path + source_files_dict.get(max(source_files_dict.keys()))
    try:
        df_data = pd.read_excel(
            last_data_path,
            sheet_name="Лист1",
            index_col=None,
        )
    except Exception as e:
        sys.exit(e)
    # print(f"используется файл:\t{last_data_path}")

    # постоянная часть
    df_constant = df_data[df_data.columns[[0, 1, 2, 5, 7, 8, 10, 12, 14]]]
    constatn_columns = [
        "id",
        "rating",
        "region_name",
        "total",
        "total_semd",
        "ouz",
        "ouz_semd",
        "private",
        "private_semd",
    ]
    df_constant.columns = constatn_columns
    df_constant = df_constant.drop(index=[0, 1, 2, 3])
    df_constant = df_constant.reset_index(drop=True)
    df_constant = df_constant.astype(
        {
            "id": int,
            "rating": int,
            "total": int,
            "total_semd": int,
            "ouz": int,
            "ouz_semd": int,
            "private": int,
            "private_semd": int,
        }
    )

    # динамическая часть
    variable_columns_numbers = [2] + [x for x in range(44, df_data.shape[1])]
    df_variable = df_data[df_data.columns[variable_columns_numbers]]
    df_variable = df_variable.drop(index=[0, 1, 2])
    df_variable.at[3, df_variable.columns[0]] = "region_name"
    df_variable.columns = df_variable.iloc[0]
    df_variable = df_variable.drop([3]).reset_index(drop=True)
    df_variable = pd.melt(
        df_variable,
        id_vars=df_variable.columns[0],
        value_vars=[df_variable.columns[x] for x in range(1, df_variable.shape[1])],
    )
    df_variable = df_variable.rename(columns={3: "date_ratio"})
    df_variable["value"] = df_variable["value"].astype("float")

    # rating_date = df_variable["date_ratio"].max().strftime("%Y-%m-%d")

    # заливка с заменой
    db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    df_constant.to_sql("semd_status", con=conn, index=False, if_exists="replace")
    df_variable.to_sql("semd_dynamic", con=conn, index=False, if_exists="replace")
    return last_data_path

In [47]:
update()

'c:/work/pkz_bot/bot_order/source_data/Рейтинг_И7_28.08.2024 в2.xlsx'

### Ответчик

In [62]:
user_name = "kinick"

def answerer(user_name):
	"""получение ответа из db"""
	db_path = os.getcwd().replace("\\", "/") + "/db_order_username.sqlite"
	conn = sqlite3.connect(db_path)
	cur = conn.cursor()
	query = """
	SELECT
		rn.reg_name_rus_etalon,
		ss.region_name AS reg_name_rating,
		ss.rating,
		COALESCE (ss.total_semd*100.0/ss.total, 0) AS total_ratio,
		COALESCE (ss.private_semd*100.0/ss.private, 0) AS private_ratio,
		COALESCE (ss.private-ss.private_semd, 0) AS private_nosemd,
		COALESCE (ss.ouz_semd*100.0/ss.ouz, 0) AS ouz_ratio,
		COALESCE (ss.ouz-ss.ouz_semd, 0) AS ouz_nosemd,
		sd.report_date
	FROM semd_status ss
	LEFT JOIN
	(
		SELECT sd.region_name, date(MAX(sd.date_ratio)) AS report_date
		FROM semd_dynamic sd
		GROUP BY sd.region_name
	) AS sd ON ss.region_name = sd.region_name
	LEFT JOIN regions_names rn
	ON rn.reg_name_rus_rating = ss.region_name
	WHERE rn.reg_name_rus_etalon IN 
	(
		SELECT ur.reg_name_rus_etalon
		FROM user_region ur
		WHERE ur.user_name = ?
	);
	"""
	query_max_rating = """
	SELECT MAX(rating)
	FROM semd_status ss;
	"""
	max_rating = cur.execute(query_max_rating).fetchall()[0][0]

	answer_array = cur.execute(query, (user_name,)).fetchall()[0]
	answer = f"""
	📈
	Уважаемый {user_name}:
		• ваш регион: {answer_array[0]};
		• отчётная дата: {answer_array[8]};
		• рейтинг региона: {answer_array[2]} (из {max_rating});
		• доля МО, передавших не менее одного СЭМД: {round(answer_array[3], 1)} %;
		• доля частных МО, передавших не менее одного СЭМД: {round(answer_array[4], 1)} %;
		• количество частных МО, не передавших ни одного СЭМД: {"нет" if answer_array[5] == 0 else answer_array[5]};
		• доля МО, подвед. ОУЗ, передавших не менее одного СЭМД: {round(answer_array[6], 1)} %;
		• количество МО, подвед. ОУЗ, не передавших ни одного СЭМД: {"нет" if answer_array[7] == 0 else answer_array[7]}.
	"""
	return answer

In [63]:
print(answerer(user_name))


	📈
	Уважаемый kinick:
		• ваш регион: Ивановская область;
		• отчётная дата: 2024-08-22;
		• рейтинг региона: 54 (из 85);
		• доля МО, передавших не менее одного СЭМД: 77.6 %;
		• доля частных МО, передавших не менее одного СЭМД: 57.1 %;
		• количество частных МО, не передавших ни одного СЭМД: 9;
		• доля МО, подвед. ОУЗ, передавших не менее одного СЭМД: 100.0 %;
		• количество МО, подвед. ОУЗ, не передавших ни одного СЭМД: нет.
	
